# Gleitkommazahlen in Python

Wenn Sie Ganzzahlen in Python dividieren (dazu genügt Division durch $1$) oder den Typ `float` explizit auf etwas anwenden, was als Gleitkommazahl interpretiert werden kann, oder wenn Sie eine Zahl mit einem Dezimalpunkt hinschreiben, entsteht eine Gleitkommazahl. Der Standardtyp für Gleitkommazahlen in Python ist `float`, was "double precision"-Gleitkommazahlen in Binärdarstellung entspricht, wie sie im Standard [IEEE 754](https://de.wikipedia.org/wiki/IEEE_754) festgelegt sind.

In [1]:
print( 5/1 )
print( float('5') )
print( float( 5 ) )
print( 5. )

5.0
5.0
5.0
5.0


Wie Sie sehen, kann man eine Gleitkommazahl auch aus einem String (hier: `'5'` oder `"5"`) erzeugen. Man kann dabei auch die *wissenschaftliche Notation* verwenden. Für $1.23\cdot 10^{-5}$ schreibt man dabei `1.23e-5`:

In [2]:
print( float( '1.23E-5' ) ) # auch ein großes "E" funktioniert
print( 1.23e-5 )

1.23e-05
1.23e-05


In numerischen Anwendungen verwenden wir das `numpy`-Paket. Der entsprechende Datentyp in `numpy` heißt `numpy.double` (da es ja "double precision" ist) oder `numpy.float64` (da es ja eine $64$-Bit-Gleitkommazahl ist):

In [3]:
import numpy as np
print( np.double(1.2), "hat den Typ", type(np.double(1.2)) )

1.2 hat den Typ <class 'numpy.float64'>


## Rundung

<div class="alert alert-block alert-info">
<b>Konvention:</b>
    <ul>
        <li> Wenn Sie in einer Aufgabe mit einer vorgegebenen dezimalen Mantissenlänge rechnen sollen, so runden Sie <b>nach jeder Rechenoperation</b> mit <b>mathematischer Rundungsregel (half-to-even)</b> im <b>dezimalen Gleitkommaformat</b>. <br>
            Runden Sie <b>"manuell"</b>, also ohne die `round()`-Funktion zu verwenden.
        <li> In Programmieraufgaben rechnen Sie mit double precision (also `float` in Python bzw. `double` in `numpy`)
            </ul>
</div>

Sowohl in Python als auch in numpy gibt es Funktionen des Namens "round", die auf eine vorgegebene Zahl von dezimalen Nachkommastellen runden. Das entspricht der Rundung im Festkomma- statt im Gleitkommaformat, und das ist der erste Grund für die oben formulierte Aufgaben-Konvention. Der zweite Grund ist, dass es subtile Fallen aufgrund des Wechselspiels zwischen dem intern verwendeten Binärformat und dem für die Ausgabe verwendeten Dezimalformat gibt und sich zudem die `round()`-Funktionen in Python und numpy manchmal unterscheiden, wie ich im Folgenden darstellen möchte.

Mit `round( x, d )` wird die Zahl $x$ auf $d$ *dezimale* **Nachkomma**stellen gerundet. Es handelt sich also wohlgemerkt um eine Rundung im Festkommaformat, nicht im Gleitkommaformat:

In [4]:
print( round(np.pi, 5) ) # np.pi ist die Zahl Pi in double precision
print( np.round(np.pi, 5) ) # np.round und round stimmen HIER überein

3.14159
3.14159


Schreibt man einfach nur `round( x )`, so wird ganzzahlig gerundet. Zunächst erhält man den Eindruck, dass Python und numpy mathematisch "half-to-even" runden:

In [5]:
# Pyton und numpy runden jeweils half-to-even:
print( round( 12.5 ) )
print( np.round( 12.5 ) )
print( round( 13.5 ) )
print( np.round( 13.5 ) )
print( round( 1.25, 1 ) )
print( np.round( 1.25, 1 ) )
print( round( 1.35, 1 ) )
print( np.round( 1.35, 1 ) )

12
12.0
14
14.0
1.2
1.2
1.4
1.4


Aber, o Schreck: In den folgenden Beispielen rundet Python anscheinend doch nicht nach der "half-to-even"-Regel, wobei zunächst unklar ist, welche Regel wirklich angewandt wird! numpy scheint sich da auf den ersten Blick besser zu verhalten:

In [6]:
print( round( 1.23445, 4) )    # kaufmännisch? half-to-odd?
print( np.round( 1.23445, 4) ) # half-to-even
print( round( 16.055, 2 ) )    # half-to-odd? Jedenfalls nicht kaufmännisch
print( np.round( 16.055, 2 ) ) # half-to-even

1.2345
1.2344
16.05
16.06


Diese subtilen Unterschiede haben ihren Grund darin, dass sich $1.23445$ sowie $16.055$ nicht  im Binärformat mit endlich vielen Nachkommastellen darstellen lässt und daher intern nicht als exakte Dezimalzahl, sondern als *gerundete* Binärzahl gespeichert sind. Python behandelt die Zahl $1.23445$ nämlich identisch mit einer Zahl, die anscheinend geringfügig größer als $1.23445$ ist.

In [7]:
print(1.234450000000000047251091928, 1.234450000000000047251091928==1.23445 )

1.23445 True


Umgekehrt behandelt Python die Zahl $16.055$ identisch mit einer Zahl, die anscheinend geringfügig kleiner als $16.055$ ist:

In [8]:
print( 16.0549999999999997, 16.0549999999999997==16.055 )

16.055 True


Insofern ist es konsequent, dass Python $1.23445$ bei Rundung auf $4$ Nachkommastellen sowie $16.055$ bei Rundung auf $2$ Nachkommastellen nicht auf gerade Endziffer rundet. Andererseits verhält sich hier numpy näher an der dezimalen Rundungsregel angewandt auf das Anzeigeformat der Zahl.

<div class="alert alert-block alert-danger">
<b>Fazit:</b><br>
    Lassen Sie in Aufgaben die Finger von round(), wenn es nicht ausdrücklich verlangt wird!
</div>

In einem weiteren Notebook werden Sie ein Paket kennen lernen, das tatsächlich im Dezimalsystem rechnet, aber für echte numerische Anwendungen zu langsam und daher eher als Illustration geeignet ist.

## Die Strukturdaten von float

Im Paket `sys` sind einige Informationen über den Leistungsumfang von `float` verfügbar, etwa über die maximal darstellbaren Zahlen oder über das Maschinenepsilon:

In [9]:
import sys
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

Man kann die einzelnen Strukturdaten über ihren Namen erhalten:

In [10]:
sys.float_info.epsilon

2.220446049250313e-16

Prüfen wir, ob dieses Epsilon wirklich das Maschinenepsilon ist: $\epsilon$ ist die kleinste obere Schranke für den relativen Rundungsfehler von Gleitkommazahlen (siehe die Definition im Skript oder auch in [Wikipedia](https://de.wikipedia.org/wiki/Maschinengenauigkeit)); wegen der half-to-even-Regel müsste also $1+\epsilon=1$ gerundet werden. Doch das ist offenbar nicht der Fall:

In [11]:
1 + sys.float_info.epsilon

1.0000000000000002

Also folgt Python offenbar der alternativen Konvention, dass das Maschinen-$\varepsilon$ der Abstand von $1$ zur nächstgrößeren darstellbaren normalisierten Gleitkommazahl ist:

In [12]:
1 + sys.float_info.epsilon/2

1.0

<div class="alert alert-block alert-danger">
<b>Halten Sie sich an die Definition aus dem Skript,</b> zumal sie mit der Definition in Wikipedia übereinstimmt.
</div>

Übrigens ist `sys.float_info.min` nicht die kleinste überhaupt darstellbare positive Zahl, denn es gibt ja noch "subnormale" Zahlen:

In [13]:
sys.float_info.min / 4

5.562684646268003e-309

Die subnormalen Zahlen zeichnen sich dadurch aus, dass sie nicht Null sind, aber ihr Kehrwert in Gleitkommaarithmetik ist $\infty$:

In [14]:
1 / (sys.float_info.min/4)

inf